In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
import pickle
import numpy as np
import time

In [11]:
# load dataset and labels
X_train = pickle.load(open('pickle_data/X_train.pickle','rb'))
y_train = pickle.load(open('pickle_data/y_train.pickle','rb'))

# load validation dataset and labels
X_validation = pickle.load(open('pickle_data/X_validation.pickle', 'rb'))
y_validation = pickle.load(open('pickle_data/y_validation.pickle', 'rb'))

print('X_train.shape =', X_train.shape)
print('y_train.shape =', y_train.shape)
print('X_validation.shape =', X_validation.shape)
print('y_validation.shape =', y_validation.shape)

X_train.shape = (1902, 50, 50, 1)
y_train.shape = (1902,)
X_validation.shape = (212, 50, 50, 1)
y_validation.shape = (212,)


In [12]:
# normalize pixel data
X_train = X_train/255.0
X_validation = X_validation/255.0

In [13]:
# define hyperparameters
conv_layer = 2
layer_size = 64
dense_layer = 0

In [14]:
# name of model to keep track of changing model architectures
NAME = '{}-conv-{}-nodes-{}-dense-{}'.format(conv_layer, layer_size, dense_layer, int(time.time()))
# create a new log of the model to analyze results
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

# define model
model = Sequential()

# 1st convolution
model.add(Conv2D(layer_size, (3,3), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# add additional conv layers
for _ in range(conv_layer-1):
    # convolution
    model.add(Conv2D(layer_size, (3,3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    # Max-Pool to reduce tensor size while retaining distict features
    model.add(MaxPooling2D(pool_size=(2,2)))

# flatten model for fully connected layers
model.add(Flatten())

# add dense layers
for _ in range(dense_layer):
    # fully-connected
    model.add(Dense(layer_size))
    model.add(Activation('relu'))

# final fully-connected layer (151 matches number of Pokemon categories)
model.add(Dense(151))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=16, epochs=20, validation_split=0.2, callbacks=[tensorboard])


Train on 1521 samples, validate on 381 samples
Epoch 1/20
1521/1521 [==============================] - 5s 3ms/sample - loss: 1.5325 - accuracy: 0.4234 - val_loss: 2.3429 - val_accuracy: 0.3123
Epoch 2/20
1521/1521 [==============================] - 4s 3ms/sample - loss: 0.9934 - accuracy: 0.6246 - val_loss: 1.9321 - val_accuracy: 0.2861
Epoch 3/20
1521/1521 [==============================] - 4s 3ms/sample - loss: 0.7414 - accuracy: 0.7232 - val_loss: 1.5759 - val_accuracy: 0.4567
Epoch 4/20
1521/1521 [==============================] - 4s 3ms/sample - loss: 0.5157 - accuracy: 0.8166 - val_loss: 1.4128 - val_accuracy: 0.4829
Epoch 5/20
1521/1521 [==============================] - 4s 3ms/sample - loss: 0.3627 - accuracy: 0.8856 - val_loss: 1.0261 - val_accuracy: 0.6115
Epoch 6/20
1521/1521 [==============================] - 4s 3ms/sample - loss: 0.3059 - accuracy: 0.9027 - val_loss: 1.2337 - val_accuracy: 0.6089
Epoch 7/20
1521/1521 [==============================] - 4s 3ms/sample - loss:

In [16]:
# evaluate the model
scores = model.evaluate(X_validation, y_validation, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 65.57%


In [17]:
# save the model
model.save('models/' + NAME)

INFO:tensorflow:Assets written to: models/2-conv-64-nodes-0-dense-1574030078/assets
